# **BERTopic** - Dynamic Topic Modeling with Twees hashtags: #manunited, #manchesterunited,#MUFC


In this tutorial we will be using Dynamic Topic Modeling with BERTopic to visualize how topics in Tweets have evolved over time. These topics will be visualized and thoroughly explored. 

## Dynamic Topic Models
Dynamic topic models can be used to analyze the evolution of topics of a collection of documents over time. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Nov 11 11:58:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
%%capture
!pip install bertopic

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
For this model we will need Tweets as Data. How to get data will be captured in a other notebook.

In this notebook we will do two DF's. One with stopwords, and one without stopwords. We will compare the result

In [ ]:
import re
import pandas as pd
from datetime import datetime

In [ ]:
tweets_no_stopwords = pd.read_csv('/content/drive/MyDrive/08 - SIT Autonomous/Data/outputs_twitter/Interim/_Post_Twitter_all_Hashtags_with_matches_clean_removed_stopwords_no_lem.csv')
#tweets_no_stopwords = tweets_no_stopwords.sample(n=500000)
#tweets_no_stopwords = tweets_no_stopwords[tweets_no_stopwords['User'] == 'ManUtd'] 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (14,15,16,17,18,19,20,21,22,24,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
tweets_no_stopwords.shape

(1075174, 30)

In [ ]:
sentiment = pd.read_csv('/content/drive/MyDrive/08 - SIT Autonomous/Data/_Post_Twitter_all_Hashtags_with_matches_clean_removed_stopwords_with_sentiment.csv')
sentiment.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (15,16,17,18,19,20,21,22,23,25,26,27,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1075174, 33)

In [ ]:
tweets_no_stopwords=pd.concat([tweets_no_stopwords,sentiment['Sentiment']], axis=1)

In [ ]:
tweets_no_stopwords= tweets_no_stopwords.dropna(subset = ['Tweet_clean'])

In [ ]:
tweets_no_stopwords=tweets_no_stopwords[tweets_no_stopwords['Sentiment']=='Negative']

In [ ]:
# create a list of all tweets
tweets_no_stopwords_lst = tweets_no_stopwords.Tweet_clean.to_list()
tweets_no_stopwords_lst[5123]
timestamps = tweets_no_stopwords.Date_Created.to_list()

In [ ]:
tweets_no_stopwords_lst[5123]

'Next weekend penalty someone else VAR SHIT GlazersOut'

# **Dynamic Topic Modeling**


## Basic Topic Model
To perform Dynamic Topic Modeling with BERTopic we will first need to create a basic topic model using all tweets. The temporal aspect will be ignored as we are, for now, only interested in the topics that reside in those tweets. 

In [ ]:
import pickle

In [ ]:
##%%time
from bertopic import BERTopic
topic_model = BERTopic(min_topic_size=35, verbose=True)
topics, _ = topic_model.fit_transform(tweets_no_stopwords_lst)


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/5682 [00:00<?, ?it/s]

2022-11-11 12:07:49,290 - BERTopic - Transformed documents to Embeddings
2022-11-11 12:11:16,116 - BERTopic - Reduced dimensionality
2022-11-11 12:11:33,035 - BERTopic - Clustered reduced embeddings


We can then extract most frequent topics:

In [ ]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,-1,62688,-1_players_manutd_game_ten
1,0,8782,0_glazers_glazersout_glazerssellmanutd_glazers...
2,1,8299,1_jong_frenkie_de_barcelona
3,2,5915,2_ronaldo_cristiano_cristianoronaldo_leave
4,3,3927,3_rashford_marcus_rashfords_martial
5,4,3257,4_maguire_harry_captain_maguireout
6,5,3058,5_antony_ajax_bid_antonys
7,6,2394,6_transfer_window_transfers_market
8,7,2243,7_casemiro_madrid_casemiros_madridxtra
9,8,2173,8_var_penalty_ref_referee


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_model.get_topic(4)

[('maguire', 0.04669517156254742),
 ('harry', 0.030094329662714085),
 ('captain', 0.022983716010993863),
 ('maguireout', 0.009849931590397435),
 ('maguires', 0.00787043555150021),
 ('captaincy', 0.0071092455583191775),
 ('armband', 0.005400361492857821),
 ('booing', 0.00505071746482278),
 ('hes', 0.004591059188981519),
 ('harrymaguire', 0.004017955604123923)]

# Save the model #

In [ ]:
# save the model to disk
filename = '/content/drive/MyDrive/08 - SIT Autonomous/Data/outputs_twitter/finalized_model_BERTopic_1M_DB_no_Stopwords_no_lem_negative_sentiment_only.sav'
pickle.dump(topic_model, open(filename, 'wb'))

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)



We can visualize the basic topics that were created with the Intertopic Distance Map. This allows us to judge visually whether the basic topics are sufficient before proceeding to creating the topics over time.

## Topics over Time
Before we start with the Dynamic Topic Modeling step, it is important that you are satisfied with the topics that were created previously. We are going to be using those specific topics as a base for Dynamic Topic Modeling. 

Thus, this step will essentially show you how the topics that were defined previously have evolved over time. 

There are a few important parameters that you should take note of, namely:

* `docs`
  * These are the tweets that we are using
* `timestamps`
  * The timestamp of each tweet/document
* `global_tuning`
  * Whether to average the topic representation of a topic at time *t* with its global topic representation
* `evolution_tuning`
  * Whether to average the topic representation of a topic at time *t* with the topic representation of that topic at time *t-1*
* `nr_bins`
  * The number of bins to put our timestamps into. It is computationally inefficient to extract the topics at thousands of different timestamps. Therefore, it is advised to keep this value below 20. 


In [ ]:
topics_over_time = topic_model.topics_over_time(docs=tweets_no_stopwords_lst, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

20it [00:47,  2.38s/it]


## Visualize Topics over Time
After having created our `topics_over_time`, we will have to visualize those topics as accessing them becomes a bit more difficult with the added temporal dimension. 

To do so, we are going to visualize the distribution of topics over time based on their frequency. Doing so allows us to see how the topics have evolved over time. Make sure to hover over any point to see how the topic representation at time *t* differs from the global topic representation. 

In [ ]:
fig=topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)
fig.show()

In [ ]:
fig.write_html("/content/drive/MyDrive/08 - SIT Autonomous/Presentation/negative_sentiment.html")